In [2]:
import numpy as np 
import pandas as pd
import nltk
import re

In [8]:
story = """Kui emme tahab kinno, siis emme läheb kinno!
Võta beebi kaasa, sõbrannad ligi ja Solaris Kino ootab teid!
NB! Emme ja Beebi Kinopäevale on ka issid väga teretulnud!
Imelises Kehrwiederi Jäätisekohvikus kinopileti esitamisel saavad emmed ja nende beedid 2 teed või 2 mahla ühe hinnaga, samuti pakutakse 10% soodustust teistele jookidele ja kookidele.
Kohvikus Komeet ootab teid spetsiaalne BEEBIMENÜÜ – klientidele vanuses 1-17 kuud. Valmistame erinevaid värskeid aed- ja puuviljamehusid ning püreesid kõige pisematele.
Lase oma sisemine sportlane välja. 
"""

In [9]:
from nltk.tokenize import sent_tokenize

In [10]:
sentences = [sent_tokenize(story)] 

In [11]:
# flatten list
sentences = [y for x in sentences for y in x]

In [15]:
sentences[:5]

['Kui emme tahab kinno, siis emme läheb kinno!',
 'Võta beebi kaasa, sõbrannad ligi ja Solaris Kino ootab teid!',
 'NB!',
 'Emme ja Beebi Kinopäevale on ka issid väga teretulnud!',
 'Imelises Kehrwiederi Jäätisekohvikus kinopileti esitamisel saavad emmed ja nende beedid 2 teed või 2 mahla ühe hinnaga, samuti pakutakse 10% soodustust teistele jookidele ja kookidele.']

In [54]:
# remove punctuations, numbers and special characters 
clean_sentences = pd.Series(sentences).str.replace("[.!?\\-]", " ") 
clean_sentences = pd.Series(clean_sentences).str.replace("^\d+\s|\s\d+\s|\s\d+$", " ")
# make alphabets lowercase 
clean_sentences = [s.lower() for s in clean_sentences]

In [55]:
clean_sentences[:5]

['kui emme tahab kinno, siis emme läheb kinno ',
 'võta beebi kaasa, sõbrannad ligi ja solaris kino ootab teid ',
 'nb ',
 'emme ja beebi kinopäevale on ka issid väga teretulnud ',
 'imelises kehrwiederi jäätisekohvikus kinopileti esitamisel saavad emmed ja nende beedid teed või mahla ühe hinnaga, samuti pakutakse 10% soodustust teistele jookidele ja kookidele ']

In [20]:

stop_words = set(line.strip() for line in open('stopwords-et.txt'))
print (stop_words)

{'mul', 'sa', 'kui', 'oled', 'mida', 'aga', 'mu', 'siis', 'ära', 'nad', 'siin', 'mulle', 'ta', 'seda', 'see', 'oma', 'et', 'on', 'selle', 'mis', 'olen', 'pole', 'jah', 'ei', 'kõik', 'mind', 'me', 'minu', 'ma', 'oli', 'nii', 'kas', 'ja', 'te', 'midagi'}


In [21]:
# function to remove stopwords 
def remove_stopwords(sen):     
    sen_new = " ".join([i for i in sen if i not in stop_words])          
    return sen_new

In [22]:
# remove stopwords from the sentences 
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
print (clean_sentences)

['emme tahab kinno, emme läheb kinno', 'võta beebi kaasa, sõbrannad ligi solaris kino ootab teid', 'nb', 'emme beebi kinopäevale ka issid väga teretulnud', 'imelises kehrwiederi jäätisekohvikus kinopileti esitamisel saavad emmed nende beedid 2 teed või 2 mahla ühe hinnaga, samuti pakutakse 10% soodustust teistele jookidele kookidele', 'kohvikus komeet ootab teid spetsiaalne beebimenüü – klientidele vanuses 1 17 kuud', 'valmistame erinevaid värskeid aed puuviljamehusid ning püreesid kõige pisematele', 'lase sisemine sportlane välja']


In [37]:
# Extract word vectors 
word_embeddings = {} 
# f = open('wiki.et.vec', encoding='utf-8') 
# firstLine = f.pop(0)
dimension = 0
with open('wiki.et.vec', encoding='utf-8') as f:
    first_line = f.readline()
    dimension = int(first_line.split()[-1])
    print (first_line)
    for line in f: 
        values = line.split() 
        word = values[0]
        #print (word)
        try:
            coefs = np.asarray(values[1:], dtype='float32') 
            assert(len(values[1:])==300)
            word_embeddings[word] = coefs
        except:
            print ("error")
        
f.close()


329987 300

error
error
error
error
error
error
error
error
error
error
error
error
error


In [42]:
print (len(list(word_embeddings.keys())))
print (dimension)

329900
300


In [43]:
sentence_vectors = [] 
for i in clean_sentences: 
  if len(i) != 0: 
    v = sum([word_embeddings.get(w, np.zeros((dimension,))) for w in  
        i.split()])/(len(i.split())+0.001) 
  else: 
    v = np.zeros((dimension,)) 
  sentence_vectors.append(v)

In [44]:
# similarity matrix 
sim_mat = np.zeros([len(sentences), len(sentences)])

In [45]:
from sklearn.metrics.pairwise import cosine_similarity

In [46]:
for i in range(len(sentences)): 
  for j in range(len(sentences)): 
    if i != j: 
      sim_mat[i][j] = cosine_similarity (sentence_vectors[i].reshape(1,dimension), sentence_vectors[j].reshape(1,dimension))[0,0]

In [47]:
import networkx as nx 
nx_graph = nx.from_numpy_array(sim_mat) 
scores = nx.pagerank(nx_graph)

In [51]:
ranked_sentences = sorted(((scores[i],s) for i,s in 
                           enumerate(sentences)), reverse=True)

n_sentences = int(np.ceil(len(ranked_sentences) ** 0.5))
# Extract top 10 sentences as the summary 
for i in range(n_sentences): 
  print(ranked_sentences[i][1])

Võta beebi kaasa, sõbrannad ligi ja Solaris Kino ootab teid!
Emme ja Beebi Kinopäevale on ka issid väga teretulnud!
Kohvikus Komeet ootab teid spetsiaalne BEEBIMENÜÜ – klientidele vanuses 1-17 kuud.
